# Running the Analysis on all Subjects

In [1]:
from IPython.display import display, HTML
from ipywidgets import interact, interactive
from py2neo import Node, Relationship, Graph
from scipy.linalg import toeplitz
from scipy.stats import binom
from tqdm import tqdm_notebook as tqdm

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle as pk
import pymania as mn

In [2]:
%matplotlib widget
graph = Graph(host="canopus.cc.gatech.edu",password='1234')
subjects = [126426, 135124, 137431, 144125, 146735, 152427, 153227, 177140, 180533, 186545,
            188145, 192237, 206323, 227533, 248238, 360030, 361234, 362034, 368753, 401422,
            413934, 453542, 463040, 468050, 481042, 825654, 911849, 917558, 992673, 558960,
            569965, 644246, 654552, 680452, 701535, 804646, 814548]
NOS = 5000
n_subjects = len(subjects)

In [3]:
def joini(lst):
    return ', '.join(str(x) for x in lst)

# Run Mania on all Subjects

In [6]:
def get_network(subjects, run_type):
    network = mn.create_project('Constantine', run_type)
    for roi in ['R'+str(xx) for xx in range(1,181)]:
        network.add_roi(roi)
    for subject in subjects:
        network.add_subject(subject)
    network.load()
    network.run()
    return network

In [7]:
vdense_network = get_network(subjects, 'vdense_temp')

In [ ]:
networks = {'vdense':vdense_network}
pk.dump(networks, open( "/net/ht140/manoj/All_Subjects_networks_R_dense.pkl", "wb"))

In [ ]:
sparse_network = get_network(subjects, 'sparse_temp')

In [1]:
networks = {'vdense':vdense_network, 'sparse':sparse_network}

NameError: name 'vdense_network' is not defined

In [2]:
pk.dump(networks, open( "/net/ht140/manoj/All_Subjects_networks_R.pkl", "wb"))

NameError: name 'networks' is not defined

In [4]:
base_dir = '/net/ht140/kamal/MANIA2_PICKLES'
network_l_sp = pk.load(open(f'{base_dir}/sparse_left_37.pk', 'rb'))
network_r_sp = pk.load(open(f'{base_dir}/sparse_right_37.pk', 'rb'))
network_l_vd = pk.load(open(f'{base_dir}/vdense_left_37.pk', 'rb'))
network_r_vd = pk.load(open(f'{base_dir}/vdense_right_37.pk', 'rb'))

# Corrected ranges containing 0
Considering only the ROI pairs for which we actually perform a distance correction, what is the fraction of ROI pairs for which the corrected ranges include 0?

In [10]:
subject = 126426
network = vdense_network

In [12]:
gr0 = {}
cw0 = {}
for subject in subjects:
    for i in range(1,181):
        for j in range(i+1, 181):
            source, target = 'L%d' % i, 'L%d' % j
            st_pair = vdense_network(subject)(source, target, pair=True)
            st1, st2 = st_pair.st1, st_pair.st2
            reg1, reg2 = st_pair.st1.regressor, st_pair.st2.regressor

            if st1.corrected_weight > 0:
                st_type = st1.get_type()
                if st_type not in gr0:
                    gr0[st_type] = []
                if st_type not in cw0:
                    cw0[st_type] = []
                gr0[st_type].append((subject, source, reg1.kind, target, reg2.kind))
                min_cw = min(st1._corrected_weights)
                if min_cw <= 0:
                    cw0[st_type].append((subject, source, reg1.kind, target, reg2.kind))
                
            if st2.corrected_weight > 0:
                st_type = st2.get_type()
                if st_type not in gr0:
                    gr0[st_type] = []
                if st_type not in cw0:
                    cw0[st_type] = []
                gr0[st_type].append((subject, target, reg2.kind, source, reg1.kind))
                min_cw = min(st2._corrected_weights)
                if min_cw <= 0:
                    cw0[st_type].append((subject, target, reg2.kind, source, reg1.kind))

In [13]:
html = '<table><tr><th>Correction Type</th><th># links min < 0</th><th>Total # > 0</th><th>Percentage</th></tr>'
for st_type, links in gr0.items():
    a, b = len(cw0[st_type]), len(gr0[st_type])
    html += '<tr><th>%s</th><td>%d</td><td>%d</td><td>%5.2f</td></tr>' % (st_type, a, b, a*100./b)
html += '</table>'
display(HTML(html))

Correction Type,# links min < 0,Total # > 0,Percentage
Independent,6078,64337,9.45
one direction,5565,86511,6.43
poolAll,347,8648,4.01


# Modify with lowest value in confidence range

In [15]:
gr0_min = {}
cw0_min = {}
for subject in subjects:
    for i in range(1,181):
        for j in range(i+1, 181):
            source, target = 'L%d' % i, 'L%d' % j
            st_pair = vdense_network(subject)(source, target, pair=True)
            st1, st2 = st_pair.st1, st_pair.st2
            reg1, reg2 = st_pair.st1.regressor, st_pair.st2.regressor

            if st1.corrected_weight > 0:
                st_type = st1.get_type()
                if st_type not in gr0_min:
                    gr0_min[st_type] = []
                if st_type not in cw0_min:
                    cw0_min[st_type] = []
                gr0_min[st_type].append((subject, source, reg1.kind, target, reg2.kind))
                if st1._corrected_weight_min <= 0:
                    cw0_min[st_type].append((subject, source, reg1.kind, target, reg2.kind))
                
            if st2.corrected_weight > 0:
                st_type = st2.get_type()
                if st_type not in gr0_min:
                    gr0_min[st_type] = []
                if st_type not in cw0_min:
                    cw0_min[st_type] = []
                gr0_min[st_type].append((subject, target, reg2.kind, source, reg1.kind))
                if st2._corrected_weight_min <= 0:
                    cw0_min[st_type].append((subject, target, reg2.kind, source, reg1.kind))

In [16]:
html = '<table><tr><th>Correction Type</th><th># links min < 0</th><th>Total # > 0</th><th>Percentage</th></tr>'
for st_type, links in gr0.items():
    a, b = len(cw0_min[st_type]), len(gr0_min[st_type])
    html += '<tr><th>%s</th><td>%d</td><td>%d</td><td>%5.2f</td></tr>' % (st_type, a, b, a*100./b)
html += '</table>'
display(HTML(html))

Correction Type,# links min < 0,Total # > 0,Percentage
Independent,749,64337,1.16
one direction,566,86511,0.65
poolAll,9,8648,0.10


# Predictability of Networks

One possible study is predicting if an edge existing is subject $i$ using all other subjects $S_{-i}$. To do this, we introduce a detection threshold $\gamma$ i.e., if edge $e$ is present in $\gamma$ fraction of subjects in $S_{-i}$ it will be present in $S_i$. Since it is a detection problem, we can study the ROC parameterized by $\gamma$. We should show that MANIA2 has a better prediction power than a fixed threshold.

In [21]:
network = vdense_network
total_connected = np.zeros((181,181))
total_connected_1 = np.zeros((181,181))

for i in range(1, 181):
    for j in range(1, 181):
        if i == j:
            continue
        source, target = 'L%d' % i, 'L%d' % j
        for sub_i in subjects:
            if network(sub_i).is_connected(source, target):
                total_connected[i,j] += 1
            if network(sub_i).is_connected(source, target, mania2=False):
                total_connected_1[i,j] += 1

In [22]:
num_correct, num_corr_m1 = [], []
for gamma in tqdm(range(1, len(subjects)), desc='Gamma'):
    n_correct, n_corr_m1 = 0, 0
    for sub_i in tqdm(subjects, desc='Subjects', leave=False):
        for i in range(1,181):
            for j in range(1, 181):
                if i == j:
                    continue
                source, target = 'L%d' % i, 'L%d' % j
                # Mania 2
                n_connected = total_connected[i,j]
                if network(sub_j).is_connected(source, target):
                    n_connected -= 1
                if network(sub_j).is_connected(source, target) and n_connected >= gamma:
                    n_correct += 1
                if (not network(sub_j).is_connected(source, target)) and n_connected < gamma:
                    n_correct += 1
                # Mania 1
                n_connected = total_connected_1[i,j]
                if network(sub_j).is_connected(source, target, False):
                    n_connected -= 1
                if network(sub_j).is_connected(source, target, False) and n_connected >= gamma:
                    n_corr_m1 += 1
                if (not network(sub_j).is_connected(source, target, False)) and n_connected < gamma:
                    n_corr_m1 += 1
    num_correct.append(n_correct)
    num_corr_m1.append(n_corr_m1)

In [ ]:
den = len(subjects)*180*179.
lists = sorted(num_correct.items())
x, y = zip(*lists)
x, y = list(x), list(y)
y = [i/den for i in y]
plt.figure()
plt.plot(x,y)
plt.grid()
plt.show()

NameError: name 'subjects' is not defined

In [ ]:
subjects = sub_train
network = vdense_network
tp, fp, fn, tn = [], [], [], []
den = len(subjects)*180*179.

for gamma in tqdm(range(1, len(subjects)), desc='Gamma'):
    n_tp, n_fp, n_fn, n_tn = 0, 0, 0, 0
    for sub_i in tqdm(subjects, desc='Subjects', leave=False):
        for i in range(1,181):
            for j in range(1, 181):
                if i == j:
                    continue
                source, target = 'L%d' % i, 'L%d' % j
                n_connected = total_connected[i,j]
                if network(sub_j).is_connected(source, target):
                    n_connected -= 1
                if n_connected >= gamma: # Actual value True
                    if network(sub_j).is_connected(source, target): # Predicted True
                        n_tp += 1
                    else: # Predicted False
                        n_fn += 1 # Predicted
                else: # Actual value False
                    if network(sub_j).is_connected(source, target): # Predicted True
                        n_fp += 1
                    else: # Predicted False
                        n_tn += 1
    n_tp, n_fp, n_fn, n_tn = n_tp/den, n_fp/den, n_fn/den, n_tn/den
    tp.append(n_tp)
    fp.append(n_fp)
    tn.append(n_tn)
    fn.append(n_fn)

In [ ]:
tpr = list(np.divide(tp, [sum(x) for x in zip(tp, fn)]))
fpr = list(np.divide(fp, [sum(x) for x in zip(fp, tn)]))
tpr.insert(0, 0)
tpr.append(1)
fpr.insert(0, 0)
fpr.append(1)
plt.figure()
plt.plot(fpr, tpr)
plt.grid()
plt.show()

# Network Density and Consistency across Subjects

In [38]:
n_connected_vd = 0
n_connected_sp = 0
den = len(subjects) * 180 * 179
den_sub = 180 * 179 / 100.

n_conn_sub_vd, n_conn_sub_sp = {}, {}
thrsh_vd, thrsh_sp = {}, {}
nar_vd, nar_sp = {}, {}
nar_vd_avg, nar_sp_avg, thrsh_vd_avg, thrsh_sp_avg = 0, 0, 0, 0

for subject in tqdm(subjects):
    n_conn_sub_vd[subject], n_conn_sub_sp[subject] = 0, 0
    thrsh_vd[subject] = vdense_network(subject).threshold2
    thrsh_sp[subject] = sparse_network(subject).threshold2
    nar_vd[subject] = mn.utils.NAR(vdense_network(subject).mania2_network)
    nar_sp[subject] = mn.utils.NAR(sparse_network(subject).mania2_network)
    nar_vd_avg += nar_vd[subject]
    nar_sp_avg += nar_sp[subject]
    thrsh_vd_avg += thrsh_vd[subject]
    thrsh_sp_avg += thrsh_sp[subject]
    for i in range(1, 181):
        for j in range(i+1, 181):
            source, target = 'L%d' % i, 'L%d' % j
            if vdense_network(subject).is_connected(source, target):
                n_connected_vd += 1
                n_conn_sub_vd[subject] += 1
            if vdense_network(subject).is_connected(target, source):
                n_connected_vd += 1
                n_conn_sub_vd[subject] += 1
            if sparse_network(subject).is_connected(source, target):
                n_connected_sp += 1
                n_conn_sub_sp[subject] += 1
            if sparse_network(subject).is_connected(target, source):
                n_connected_sp += 1
                n_conn_sub_sp[subject] += 1
    n_conn_sub_vd[subject] /= den_sub
    n_conn_sub_sp[subject] /= den_sub

nar_vd_avg /= float(len(subjects))
nar_sp_avg /= float(len(subjects))
thrsh_vd_avg /= float(len(subjects))
thrsh_sp_avg /= float(len(subjects))

In [39]:
density_vd = n_connected_vd * 100. / den
density_sp = n_connected_sp * 100. / den
print('Density of Very Dense Network: %g%%' % density_vd)
print('Density of Sparse Network: %g%%' % density_sp)

Density of Very Dense Network: 26.4761%
Density of Sparse Network: 11.4128%


In [40]:
html = '<table><tr><th rowspan="2" align="center" >Subject</th><th colspan="3">Sparse</th><th colspan="3">Very Dense</th></tr><tr>'
for i in range(2):
    html += '<th>Density</th><th>Threshold</th><th>NAR</th>'
html += '</tr>'
for subject in subjects:
    html += "<tr><td>%d</td>" % subject
    html += "<td>%5.2f%%</td><td>%d</td><td>%5.3f</td>" % (n_conn_sub_sp[subject], thrsh_sp[subject], nar_sp[subject])
    html += "<td>%5.2f%%</td><td>%d</td><td>%5.3f</td>" % (n_conn_sub_vd[subject], thrsh_vd[subject], nar_vd[subject])
    html += "</tr>"
html += f'<tr><th>Mean</th>'
html += "<th>%5.2f%%</th><th>%d</th><th>%5.3f</th>" % (density_sp, thrsh_sp_avg, nar_sp_avg)
html += "<th>%5.2f%%</th><th>%d</th><th>%5.3f</th>" % (density_vd, thrsh_vd_avg, nar_vd_avg)
display(HTML(html))

# Mania Plots

In [42]:
plot_manias_cache = dict()

In [56]:
def plot_manias(subject):
    subject = int(subject)
    if subject not in plot_manias_cache:
        sparse_sub, vdense_sub = sparse_network(subject), vdense_network(subject)
        _,den_1,nar_1,t_1 = mn.utils.networks.mania_on_mat(vdense_sub.matrix1)
        _,den_s,nar_s,t_s = mn.utils.networks.mania_on_mat(sparse_sub.matrix2)
        _,den_vd,nar_vd,t_vd = mn.utils.networks.mania_on_mat(vdense_sub.matrix2)
        threshold_1 = vdense_sub.threshold1
        threshold_s, threshold_vd = sparse_sub.threshold2, vdense_sub.threshold2
        idx_1, idx_vd, idx_s = np.argmin(nar_1), np.argmin(nar_vd), np.argmin(nar_s)
        density_1, density_vd, density_s = den_1[idx_1], den_vd[idx_vd], den_s[idx_s]
        nar_value_1, nar_value_vd, nar_value_s = nar_1[idx_1], nar_vd[idx_vd], nar_s[idx_s]
        d = dict()
        d['den_1'], d['den_s'], d['den_vd'] = den_1, den_s, den_vd
        d['nar_1'], d['nar_s'], d['nar_vd'] = nar_1, nar_s, nar_vd
        d['density_1'], d['density_s'], d['density_vd'] = density_1, density_s, density_vd
        d['nar_value_1'], d['nar_value_s'], d['nar_value_vd'] = nar_value_1, nar_value_s, nar_value_vd
        d['t_1'], d['t_s'], d['t_vd'] = t_1, t_s, t_vd
        d['threshold_1'], d['threshold_s'], d['threshold_vd'] = threshold_1, threshold_s, threshold_vd
        plot_manias_cache[subject] = d
    else:
        d = plot_manias_cache[subject]
        den_1, den_s, den_vd = d['den_1'], d['den_s'], d['den_vd']
        nar_1, nar_s, nar_vd = d['nar_1'], d['nar_s'], d['nar_vd']
        density_1, density_s, density_vd = d['density_1'], d['density_s'], d['density_vd']
        nar_value_1, nar_value_s, nar_value_vd = d['nar_value_1'], d['nar_value_s'], d['nar_value_vd']
        t_1, t_s, t_vd = d['t_1'], d['t_s'], d['t_vd']
        threshold_1, threshold_s, threshold_vd = d['threshold_1'], d['threshold_s'], d['threshold_vd']
    
    fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(10,6))
    ax[0].plot(den_1,nar_1,'b-',lw=2,label='MANIA 1')
    ax[0].plot(den_s,nar_s,'g-',lw=2,label='Sparse')
    ax[0].plot(den_vd,nar_vd,'m-',lw=2,label='Very Dense')
    ax[0].axvline(density_1, 0, 1, color='b', linestyle='dashed', lw=1)
    ax[0].axvline(density_s, 0, 1, color='g', linestyle='dashed', lw=1)
    ax[0].axvline(density_vd, 0, 1, color='m', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_1, 0, 1, color='b', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_s, 0, 1, color='g', linestyle='dashed', lw=1)
    ax[0].axhline(nar_value_vd, 0, 1, color='m', linestyle='dashed', lw=1)
    ax[0].set_xlabel('Density')
    ax[0].set_ylabel('NAR')
    ax[1].plot(t_1,den_1,'b-',lw=2,label='MANIA 1')
    ax[1].plot(t_s,den_s,'g-',lw=2,label='Sparse')
    ax[1].plot(t_vd,den_vd,'m-',lw=2,label='Dense')
    ax[1].set_xlabel('Threshold')
    ax[1].set_ylabel('Density')
    ax[1].axvline(threshold_1, color='b', linestyle='dashed', lw=1)
    ax[1].axvline(threshold_s, color='g', linestyle='dashed', lw=1)
    ax[1].axvline(threshold_vd, color='m', linestyle='dashed', lw=1)
    ax[1].axhline(density_1, 0, 1, color='b', linestyle='dashed', lw=1)
    ax[1].axhline(density_s, 0, 1, color='g', linestyle='dashed', lw=1)
    ax[1].axhline(density_vd, 0, 1, color='m', linestyle='dashed', lw=1)
    plt.suptitle(f'Subject: {subject}')
    plt.legend()
    plt.savefig(f'Images/{subject}.eps')
    plt.show()

In [57]:
interact(plot_manias, subject=[str(x) for x in subjects])

interactive(children=(Dropdown(description='subject', options=('126426', '135124', '137431', '144125', '146735…

<function __main__.plot_manias(subject)>

In [58]:
for subject in subjects:
    plot_manias(subject)

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

...

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

# Analysis

In [1]:
sub_train = [126426, 137431, 144125, 146735, 152427, 153227, 177140, 180533, 186545, 188145]

# Histogram of f(e)

In [13]:
def is_corrected(st_type, is_sparse):
    if is_sparse:
        return st_type in ['Independent']
    else:
        return st_type in ['Independent', 'direction1', 'direction2', 'poolAll', 'poolEnvelopes', 'one direction']

def is_corrected_sparse(row):
    return is_corrected(row['ST_Type'], True)

def is_corrected_not_sparse(row):
    return is_corrected(row['ST_Type'], False)

In [5]:
total_connected_l_sp = np.zeros((181,181))
total_connected_r_sp = np.zeros((181,181))
total_connected_l_vd = np.zeros((181,181))
total_connected_r_vd = np.zeros((181,181))
total_connected_l_1 = np.zeros((181,181))
total_connected_r_1 = np.zeros((181,181))

for i in range(1, 181):
    for j in range(1, 181):
        if i == j:
            continue
        source_l, target_l = 'L%d' % i, 'L%d' % j
        source_r, target_r = 'R%d' % i, 'R%d' % j
        for sub_i in subjects:
            if network_l_sp(sub_i).is_connected(source_l, target_l):
                total_connected_l_sp[i,j] += 1
            if network_r_sp(sub_i).is_connected(source_r, target_r):
                total_connected_r_sp[i,j] += 1
            if network_l_vd(sub_i).is_connected(source_l, target_l):
                total_connected_l_vd[i,j] += 1
            if network_r_vd(sub_i).is_connected(source_r, target_r):
                total_connected_r_vd[i,j] += 1
            if network_l_vd(sub_i).is_connected(source_l, target_l, mania2=False):
                total_connected_l_1[i,j] += 1
            if network_r_vd(sub_i).is_connected(source_r, target_r, mania2=False):
                total_connected_r_1[i,j] += 1

In [20]:
def get_counts(network, hemisphere, is_sparse):
    is_dc = np.zeros((181,181), dtype=bool)
    num_connected = np.zeros((181,181), dtype=int)

    for i in range(1,181):
        for j in range(1,181):
            if i == j:
                continue
            source, target = '%s%d' % (hemisphere, i), '%s%d' % (hemisphere, j)
            dc = False
            n_conn = 0
            n_dc = 0
            for sub in subjects:
                st = network(sub)(source, target, pair=False)
                st._mania_loaded = True
                st._threshold2 = np.log(network(sub).threshold2/5000.)
                if is_corrected(st.get_type(), is_sparse=is_sparse):
                    n_dc += 1
                if st.isConnected():
                    num_connected[i,j] += 1
            if n_dc > 0:
                is_dc[i,j] = True

    count_dc = np.zeros(len(subjects) + 1, dtype=int)
    count_nc = np.zeros(len(subjects) + 1, dtype=int)

    for i in range(1,181):
        for j in range(1,181):
            if i == j:
                continue
            if is_dc[i,j]:
                count_dc[num_connected[i,j]] += 1
            else:
                count_nc[num_connected[i,j]] += 1
    return (is_dc, num_connected, count_dc, count_nc)

In [24]:
def plot_counts(count_dc, count_nc, title, out_file=None):
    plt.figure()
    b1 = plt.bar(range(len(subjects) + 1), count_dc)
    b2 = plt.bar(range(len(subjects) + 1), count_nc)
    plt.xlabel('#Subjects with Connection present')
    plt.ylabel('# Links')
    plt.title(title)
    plt.legend((b1[0], b2[0]), ('Distance Corrected', 'Not Distance Corrected'))
    plt.grid()
    if out_file is not None:
        plt.savefig(out_file)
    plt.show()

In [26]:
is_dc_l_sp, num_connected_l_sp, count_dc_l_sp, count_nc_l_sp = get_counts(network_l_sp, 'L', True)
is_dc_r_sp, num_connected_r_sp, count_dc_r_sp, count_nc_r_sp = get_counts(network_r_sp, 'R', True)
is_dc_l_vd, num_connected_l_vd, count_dc_l_vd, count_nc_l_vd = get_counts(network_l_vd, 'L', False)
is_dc_r_vd, num_connected_r_vd, count_dc_r_vd, count_nc_r_vd = get_counts(network_r_vd, 'R', False)

In [27]:
plot_counts(count_dc_l_sp, count_nc_l_sp, 'Left Hemisphere - Sparse Network', 'Images/All_Subjects/Counts_L_sp.eps')
plot_counts(count_dc_r_sp, count_nc_r_sp, 'Right Hemisphere - Sparse Network', 'Images/All_Subjects/Counts_R_sp.eps')
plot_counts(count_dc_l_vd, count_nc_l_vd, 'Left Hemisphere - Dense Network', 'Images/All_Subjects/Counts_L_vd.eps')
plot_counts(count_dc_r_vd, count_nc_r_vd, 'Right Hemisphere - Dense Network', 'Images/All_Subjects/Counts_R_vd.eps')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

# F Score

In [54]:
def plot_f_score(network, hemisphere, is_sparse, title, out_file=None):
    f = np.zeros((181,181))
    F1_all = dict()
    n_sub = len(subjects)
    
    for subject in subjects:
        counter = 0
        for i in range(1,181):
            for j in range(1,181):
                if i == j:
                    continue
                source, target = '%s%d' % (hemisphere, i), '%s%d' % (hemisphere, j)
                st = network(subject)(source, target, pair=False)
                st._mania_loaded = True
                st._threshold1 = np.log(network(subject).threshold1/5000.)
                st._threshold2 = np.log(network(subject).threshold2/5000.)
                if st.isConnected():
                    f[i,j] += 1.
    for i in range(1,181):
        for j in range(1,181):
            f[i,j] /= n_sub

    deltas = [(x+0.5)/n_sub for x in range(int(n_sub/2))]
    counts_all = dict()

    plt.figure(figsize=(7,10))
    for sub in subjects:
        F1, counts = [], []
        for delta in deltas:
            TP, FP, TN, FN = 0., 0., 0., 0.
            counter = 0
            for i in range(1,181):
                for j in range(1,181):
                    if i == j:
                        continue
                    source, target = '%s%d' % (hemisphere, i), '%s%d' % (hemisphere, j)

                    st = network(sub)(source, target, pair=False)
                    st._mania_loaded = True
                    st._threshold2 = np.log(network(sub).threshold2/5000.)
                    if is_corrected(st.get_type(), is_sparse=is_sparse):
                        c = 1
                    else:
                        continue

                    if f[i,j] > 0.5 + delta:
                        counter += 1
                        if st.isConnected():
                            TP += 1
                        else:
                            FN += 1
                    elif f[i,j] < 0.5 - delta:
                        counter += 1
                        if st.isConnected():
                            FP += 1
                        else:
                            TN += 1
            precision = TP / (TP + FP)
            recall = TP / (TP + FN)
            F1.append(2 * (precision * recall) / (precision + recall))
            counts.append(counter)
        F1_all[sub] = F1
        counts_all[sub] = counts
        plt.subplot(211)
        plt.plot(deltas, F1, label=str(sub))
        plt.subplot(212)
        plt.plot(deltas, counts, label=str(sub))
    plt.subplot(211)
    plt.xlabel('Delta')
    plt.ylabel('F1 Score')
    plt.title(title)
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.grid()

    plt.subplot(212)
    plt.xlabel('Delta')
    plt.ylabel('Number of Links')
    plt.grid()

    if out_file is not None:
        plt.savefig(out_file)
    plt.show()

In [58]:
plot_f_score(network_l_sp, 'L', True, 'F1 Score - Left - Sparse', 'Images/All_Subjects/F1_L_sp.eps')
plot_f_score(network_r_sp, 'R', True, 'F1 Score - Right - Sparse', 'Images/All_Subjects/F1_R_sp.eps')
plot_f_score(network_l_vd, 'L', False, 'F1 Score - Left - Dense', 'Images/All_Subjects/F1_L_vd.eps')
plot_f_score(network_r_vd, 'R', False, 'F1 Score - Right - Dense', 'Images/All_Subjects/F1_R_vd.eps')

/net/ht140/manoj/Softwares/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/net/ht140/manoj/Softwares/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()